In [ ]:
# Third-party
import astropy.table as at
from astropy.io import ascii
import astropy.coordinates as coord
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as pl
import numpy as np
pl.style.use('apw-notebook')
%matplotlib inline

In [ ]:
k2 = ascii.read("../data/catalogs/k2targets.csv")

In [ ]:
k2_c = coord.SkyCoord(ra=k2['rastr'], dec=k2['decstr'])

In [ ]:
len(k2_c)

## LINEAR

In [ ]:
linear = ascii.read("../data/catalogs/LINEAR_RRab.txt")

In [ ]:
linear_c = coord.SkyCoord(ra=linear['ra']*u.degree,
                          dec=linear['dec']*u.degree)

In [ ]:
idx,sep,_ = coord.match_coordinates_sky(linear_c, k2_c)
linear_k2 = k2[idx[(sep < 2*u.arcsecond)]]
len(linear_k2)

## NSVS

In [ ]:
nsvs = ascii.read("../data/catalogs/nsvs.tsv", delimiter='\t')

In [ ]:
nsvs_c = coord.SkyCoord(ra=nsvs['RAJ2000']*u.degree,
                        dec=nsvs['DEJ2000']*u.degree)

In [ ]:
idx,sep,_ = coord.match_coordinates_sky(nsvs_c, k2_c)
nsvs_k2 = k2[idx[(sep < 2*u.arcsecond)]]
len(nsvs_k2)

## QUEST

In [ ]:
quest = ascii.read("../data/catalogs/quest_vivas2004_phot.tsv", delimiter='\t')

In [ ]:
quest_c = coord.SkyCoord(ra=quest['_RAJ2000']*u.degree, 
                         dec=quest['_DEJ2000']*u.degree)

In [ ]:
idx,sep,_ = coord.match_coordinates_sky(quest_c, k2_c)
quest_k2 = k2[idx[(sep < 2*u.arcsecond)]]
len(quest_k2)

## Stripe 82

In [ ]:
s82 = ascii.read("../data/catalogs/stripe82-rrlyrae.txt")

In [ ]:
s82_c = coord.SkyCoord(ra=s82['ra']*u.degree, 
                       dec=s82['dec']*u.degree)

In [ ]:
idx,sep,_ = coord.match_coordinates_sky(s82_c, k2_c)
s82_k2 = k2[idx[(sep < 2*u.arcsecond)]]
len(s82_k2)

## CSS

In [ ]:
css = np.genfromtxt("../data/catalogs/css.txt", dtype=None, names=True, usecols=[1,2])

In [ ]:
css_c = coord.SkyCoord(ra=css['RA']*u.degree, 
                       dec=css['Dec']*u.degree)

In [ ]:
idx,sep,_ = coord.match_coordinates_sky(css_c, k2_c)
css_k2 = k2[idx[(sep < 2*u.arcsecond)]]
len(css_k2)

---

In [ ]:
all_k2 = at.vstack([linear_k2, nsvs_k2, quest_k2, s82_k2, css_k2])
all_k2 = at.unique(all_k2, keys='epic_number')

In [ ]:
len(all_k2)

In [ ]:
all_k2_c = coord.SkyCoord(ra=all_k2['rastr'], dec=all_k2['decstr'])
all_k2['ra'] = all_k2_c.ra.degree
all_k2['dec'] = all_k2_c.dec.degree

In [ ]:
# np.savetxt("../data/matched.txt", np.array(all_k2['epic_number']).reshape(-1,1), fmt='%d')
ascii.write(all_k2['epic_number','ra','dec'], "../data/matched.txt", delimiter=',')

In [ ]:
all_k2[:2]

In [ ]:
wget_cmd = "wget -q http://archive.stsci.edu/missions/hlsp/k2sff/c{}/{}00000/{}/hlsp_k2sff_k2_lightcurve_{}-c{}_kepler_v1_llc.fits\n"

In [ ]:
lines = []
for row in all_k2:
    campaign = row['k2_campaign_str']
    if campaign != 'E':
        campaign = '{:02d}'.format(int(campaign))
    else:
        campaign = 'et'

    str_epic = "{:09d}".format(row['epic_number'])
    ffour = str_epic[:4]
    lfive = str_epic[4:]

    lines.append(wget_cmd.format(campaign, ffour, lfive, str_epic, campaign))
    
with open("../data/light_curves/batch_wget.sh","w") as f:
    f.writelines(lines)